# Neural networks in PyTorch
Neural networks tend to have a huge number of layers. This is why they are called "deep".  
It is possible to construct a neural network with the matrix of weights, but it is difficult and cumbersome.  
PyTorch has a very nice package which permits to build large neural networks `nn`.  

**Note:** to download the dataset we used `wget` which cannot be installed properly using the commands `conda` or `pip3`, 
so we downloaded the .exe version by the [wget site](https://eternallybored.org/misc/wget/) and we put it in a folder listed in the `Path` environment variable (we chose the path to miniconda).

In [10]:
import wget
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

. 2,05M
 19850K .......... .......... .......... .......... .......... 1,93M
 19900K .......... .......... .......... .......... .......... 1,91M
 19950K .......... .......... .......... .......... .......... 1,69M
 20000K .......... .......... .......... .......... .......... 2,25M
 20050K .......... .......... .......... .......... .......... 2,55M
 20100K .......... .......... .......... .......... .......... 2,33M
 20150K .......... .......... .......... .......... .......... 2,36M
 20200K .......... .......... .......... .......... .......... 2,39M
 20250K .......... .......... .......... .......... .......... 2,60M
 20300K .......... .......... .......... .......... .......... 2,48M
 20350K .......... .......... .......... .......... .......... 2,42M
 20400K .......... .......... .......... .......... .......... 2,61M
 20450K .......... .......... .......... .......... .......... 2,41M
 20500K .......... .......... .......... .......... .......... 2,49M
 20550K .......... .......

In [6]:
# Import necessary packages

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import torch

import matplotlib.pyplot as plt

Now, we are going to create a neural network for classify digits in 10 classes which are in the dataset MNIST.   
Each image is 28x28 pixels.  
First, we want to get our dataset which is available thanks to `torchvision` package.  
The following code is able to divide the dataset in training and test set.

In [14]:
import torch
from torchvision import datasets, transforms

# define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

# Download and load the training data
trainset = datasets.MNIST('./', download=False, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

We have the training data loaded in `trainloader` and we make that an iterator with `iter(trainloader)`.  

We will use this for looping in the training data, as follows:  
`for image,label in trainloader:  
do something with images and labels`  

We have created `trainloader` with `batch_size=64` and `shuffle=True`.  
The batch size is the number of images we use for training (one iteration on dataloader). And `shuffle=True` tells to shuffle  
the dataset every time we start going through the data loader again.  

In the following chunck of code we take only the first batch, so we can check the data.  
So, we can see that the images are tensors and that one tensor has size of `[64, 1, 28, 28]` which means that,  
as expected, we have a tensor of 64 images of 1 channel and width and height 28

In [15]:
dataiter = iter(trainloader)
images, labels = dataiter.next()
print(type(images))
print(images.shape)
print(labels.shape)

<class 'torch.Tensor'>
torch.Size([64, 1, 28, 28])
torch.Size([64])
